# FSL Pipeline 


In [22]:
import os, glob
from IPython.core import display as ICD
from subprocess import check_output
import subprocess
import pandas as pd
import re

pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', None)


## Check available data in preprocessed/

Example subject directory holding preprocessed data:  

`bids/derivatives/preprocessed/sub-001
├── anat
├── ses-1
│   └── func
│       └── motion_assessment
│           └── motion_parameters
└── ses-2
    └── func
        └── motion_assessment
            └── motion_parameters`

### Run the code below to view a report of the preprocessed/ folder 

In [4]:
def check_files(dir_path, sub_id,session, data_dict):
    funcs = glob.glob(os.path.join(dir_path, sub_id, session, "func/*brain.nii.gz"))
    func_ct = len(funcs)
    data_dict[sub_id][session]["func_ct"] = func_ct

    anat = glob.glob(os.path.join(dir_path, sub_id, "anat/*"))
    anat_ct = len(anat) 
    data_dict[sub_id][session]["anat_ct"] = anat_ct


    exp = "{}_{}*.txt".format(sub_id, session)
    onsets = glob.glob('/projects/niblab/bids_projects/Experiments/bro/bids/derivatives/onsets/output_onsets/%s'%exp)
    onset_ct = len(onsets)
    data_dict[sub_id][session]["onset_ct"] = onset_ct

    
    confounds = glob.glob(os.path.join(dir_path, sub_id, session, "func/motion_assessment/*.txt"))
    confound_ct = len(confounds) 
    data_dict[sub_id][session]["confound_ct"] = confound_ct

    
    mocos = glob.glob(os.path.join(dir_path, sub_id, session, "func/motion_assessment/motion_parameters/*.txt"))
    moco_ct = len(mocos) 
    data_dict[sub_id][session]["moco_ct"] = moco_ct
        
        

In [7]:
print("\n[INFO] BRO PREPROCESSED DATA REPORT \n")

preproc_path="/projects/niblab/bids_projects/Experiments/bro/bids/derivatives/preprocessed"
sessions = ['ses-1', 'ses-2']
data_dict = {}



for session in sessions:
    subjects = sorted(glob.glob(os.path.join(preproc_path, 'sub-*', session)))
    sub_ids = [x.split("/")[-2] for x in subjects]
    subjects_ct = len(subjects)
    print("[INFO] {} subjects found count: {}".format(session, subjects_ct))

    for sub_id in sub_ids:
        if sub_id not in data_dict:
            data_dict[sub_id] = {}
            
        if session not in data_dict[sub_id]:
            data_dict[sub_id][session] = {}
            
        
        check_files(preproc_path, sub_id, session, data_dict)

df = pd.concat({k: pd.DataFrame(v).T for k, v in data_dict.items()}, axis=0)

ses1_df = df.xs('ses-1', level=1)
ses1_good_df = ses1_df[(ses1_df.T != 0).all()]
ses1_empty_df = ses1_df[(ses1_df.T == 0).any()]



ses2_df = df.xs('ses-2', level=1)
ses2_good_df = ses2_df[(ses2_df.T != 0).all()]
ses2_empty_df = ses2_df[(ses2_df.T == 0).any()]


print("\n[INFO] session 1 ready subjects: {} \n{}".format(len(ses1_good_df.index.values),ses1_good_df.index.values))
ICD.display(ses1_good_df.head())
print("\n[INFO] session 1 need processing subjects: {} \n{}".format(len(ses1_empty_df.index.values),ses1_empty_df.index.values))
ICD.display(ses1_empty_df.head())
print("\n[INFO] session 2 ready subjects: {} \n{}".format(len(ses2_good_df.index.values),ses2_good_df.index.values))
ICD.display(ses2_good_df.head())
print("\n[INFO] session 2 need processing subjects: {} \n{}".format(len(ses2_empty_df.index.values),ses2_empty_df.index.values))
ICD.display(ses2_empty_df.head())





[INFO] BRO PREPROCESSED DATA REPORT 

[INFO] ses-1 subjects found count: 54
[INFO] ses-2 subjects found count: 54

[INFO] session 1 ready subjects: 23 
['sub-020' 'sub-022' 'sub-025' 'sub-026' 'sub-028' 'sub-029' 'sub-030'
 'sub-032' 'sub-033' 'sub-035' 'sub-036' 'sub-037' 'sub-038' 'sub-039'
 'sub-040' 'sub-041' 'sub-043' 'sub-044' 'sub-045' 'sub-046' 'sub-047'
 'sub-052' 'sub-053']


,func_ct,anat_ct,onset_ct,confound_ct,moco_ct
sub-020,3,1,18,6,19
sub-022,5,1,36,10,31
sub-025,5,1,36,10,31
sub-026,5,1,36,10,31
sub-028,5,1,36,10,31



[INFO] session 1 need processing subjects: 31 
['sub-001' 'sub-002' 'sub-003' 'sub-004' 'sub-006' 'sub-007' 'sub-008'
 'sub-009' 'sub-010' 'sub-011' 'sub-012' 'sub-013' 'sub-014' 'sub-015'
 'sub-016' 'sub-017' 'sub-018' 'sub-019' 'sub-021' 'sub-023' 'sub-027'
 'sub-031' 'sub-034' 'sub-042' 'sub-048' 'sub-049' 'sub-050' 'sub-051'
 'sub-054' 'sub-055' 'sub-056']


,func_ct,anat_ct,onset_ct,confound_ct,moco_ct
sub-001,0,1,0,0,0
sub-002,0,1,0,0,0
sub-003,5,1,0,10,31
sub-004,5,1,0,10,31
sub-006,0,0,0,0,0



[INFO] session 2 ready subjects: 21 
['sub-004' 'sub-022' 'sub-025' 'sub-026' 'sub-027' 'sub-028' 'sub-029'
 'sub-030' 'sub-032' 'sub-033' 'sub-036' 'sub-038' 'sub-039' 'sub-040'
 'sub-041' 'sub-044' 'sub-045' 'sub-046' 'sub-047' 'sub-052' 'sub-053']


,func_ct,anat_ct,onset_ct,confound_ct,moco_ct
sub-004,5,1,36,10,30
sub-022,5,1,36,10,30
sub-025,5,1,36,10,30
sub-026,5,1,36,10,30
sub-027,5,1,27,10,30



[INFO] session 2 need processing subjects: 33 
['sub-001' 'sub-002' 'sub-003' 'sub-006' 'sub-007' 'sub-008' 'sub-009'
 'sub-010' 'sub-011' 'sub-012' 'sub-013' 'sub-014' 'sub-015' 'sub-016'
 'sub-017' 'sub-018' 'sub-019' 'sub-020' 'sub-021' 'sub-023' 'sub-031'
 'sub-034' 'sub-035' 'sub-037' 'sub-042' 'sub-043' 'sub-048' 'sub-049'
 'sub-050' 'sub-051' 'sub-054' 'sub-055' 'sub-056']


,func_ct,anat_ct,onset_ct,confound_ct,moco_ct
sub-001,5,1,0,10,30
sub-002,5,1,0,10,30
sub-003,4,1,0,8,24
sub-006,0,0,0,0,0
sub-007,5,1,0,10,30


## FSL level1 on Bro data

### Write level 1 models  
* **Model designs:** [google drive link](https://docs.google.com/spreadsheets/d/1bj3it16jW8lASIGgL9TIAsg0x2XuOT1JL71rTaEJ3aw/edit#gid=1860969306)   
* **Design files on RENCI:** /projects/niblab/bids_projects/Experiments/bro/bids/derivatives/design_files

In [88]:
def make_file(sub, sub_path, sess, main_dict, task, deriv_dir, design_file):
    print("[INFO] writing {} design file...".format(sub))
    
    fsf_template = os.path.join(deriv_dir,'design_files/%s'%design_file)
    #for sess_id in sessions:


    if task == "resting":
        pass

    else:
        for key in main_dict[sub]:
            if key != "ANAT":
                run = key
                #print(run)
                outpath = os.path.join(deriv_dir, 'preprocessed', sub,sess, 'analysis', "feat1")
                
                # if MAKE DIRS flag true
                if not os.path.exists(outpath):
                    os.makedirs(outpath)


                with open(fsf_template, 'r') as infile:
                    #print("Opening template file {}".format(fsf_template))
                    tempfsf = infile.read()

                    #  fill in tempfsf file with parameters
                    tempfsf = tempfsf.replace("OUTPUT", main_dict[sub][run]["OUTPUT"])
                    tempfsf = tempfsf.replace("FUNCTIONAL", main_dict[sub][run]["FUNC"])
                    tempfsf = tempfsf.replace("CONFOUND", main_dict[sub][run]['CONFOUND'])
                    tempfsf = tempfsf.replace("VOL", main_dict[sub][run]['VOL'])


                    # loop through keys in dict to find EVs and MOCOs
                    for key in main_dict[sub][run]:


                        # Fill in EVS
                        if re.match(r'EV', key):
                            ev_name= "{}_file".format(key.replace("EV_", ""))
                            ev = main_dict[sub][run][key]
                            tempfsf = tempfsf.replace(ev_name, ev)
                            #print(ev_name)
                        if re.match(r'moco', key):
                            moco_file = main_dict[sub][run][key]
                            moco_id = moco_file.split("/")[-1].split("_")[4].split(".")[0]+"_file"
                            tempfsf = tempfsf.replace(moco_id, moco_file)
                            #print(moco_id)

                    fsf_outfile = 'task-%s_run-%s.fsf' % (task,'{0:02d}'.format(int(run)) )
                    print(os.path.join(outpath, fsf_outfile))
                    #print(tempfsf)
                    with open(os.path.join(outpath, fsf_outfile), 'w') as outfile: #os.path.join(outpath,
                        outfile.write(tempfsf)
                    outfile.close()
                infile.close()

In [8]:
def fill_dict(sub, sub_path, main_dict, task, sess, evs, all_runs):
    
    
    #print("SUBJECT: %s \t TASK: %s \nPATH: %s"% (sub, task, sub_path))

    # only specified sessions
    #for sess_id in sessions:

    if task == 'resting':
        # case for no runs, only task (i.e. resting)
        pass
    else:
    # 2 cases: individual/given runs or all runs found

        # case 1: if flag false, grab all available runs found
        if all_runs == True:
            funcs_found = glob.glob(os.path.join(sub_path, 'func',
                                         "%s_%s_task-%s_run-*preproc*brain.nii.gz" % (sub,sess,task)))
            #print(funcs_found)
            runs=[x.split("/")[-1].split("_")[3].split("-")[1] for x in funcs_found]
            #print(runs)
            for run in runs:
                main_dict[sub][run] = {}
            #print("Dictionary initialized as: {}".format(main_dict[sub]))

            for func in funcs_found:
                x = int(run)
                run=func.split("/")[-1].split("_")[3].split("-")[1]
                #print(run)
                
                # SET OUTPUT PATH FOR FEAT DIRECTORY
                output_path=os.path.join(sub_path, 'analysis', 'feat1', 'task-%s_run-%s' %(task, run))


                # SET CONFOUND
                # sub-004_ses-2_task-training_run-2_space-MNI152NLin2009cAsym_desc-preproc_confound.txt

                confound = os.path.join(sub_path, 'func', 'motion_assessment',
                                 '%s_%s_task-%s_run-%s_space-MNI152NLin2009cAsym_desc-preproc_confound.txt'%(sub,sess, task, run))

                # SET ANAT
                #anat = os.path.join(sub_path.strip("{}/".format(sess)), 'anat', 'highres.nii.gz')





                # FILL DICTIONARY
                #main_dict[sub]['ANAT'] = anat
                main_dict[sub][run]['OUTPUT'] = output_path
                scan = func.split(".")[0]
                main_dict[sub][run]['FUNC'] = scan
                vol = check_output(['fslnvols', scan])
                vol = vol.decode('utf-8')
                vol = vol.strip('\n')
                main_dict[sub][run]['VOL'] = vol
                main_dict[sub][run]['CONFOUND'] = confound



                # TRS FROM NIFTI -- this value will always be 2, therefore we only run the check once
                trs = check_output(['fslval', '%s' % (scan), 'pixdim4', scan])
                trs = trs.decode('utf-8')
                trs = trs.strip('\n')
                # print("TRs: ", trs)

                main_dict[sub][run]['TR'] = trs


                # SET MOTION PARAMETERS
                for i in range(6):
                    motcor = os.path.join(sub_path, 'func', 'motion_assessment', 'motion_parameters',
                                      '%s_%s_task-%s_run-%s_moco%s.txt' % (sub, sess,task, run, i))
                    main_dict[sub][run]['moco%i' % i] = motcor


                # SET EVS
                # Loop through the given EVs and add the corresponding file to the dictionary

                ctr = 0
                onset_path = "/projects/niblab/bids_projects/Experiments/bro/bids/derivatives/onsets/output_onsets"
                for ev_name in evs:
                    # print(item)
                    ctr = ctr + 1
                    #sub-037_ses-1_training_run-1_h2o.txt
                    ev = os.path.join(onset_path,  '%s_%s_task-%s_run-%s_%s.txt' % (sub, sess, task, run, ev_name))
                    
                    #print(ev)
                    # print("EV: ", ev)
                    main_dict[sub][run]['EV_%s' % ev_name] = ev
                    
                    
                    

In [9]:
def setup_lvl1_design_files(sub_ids):
    #set_paths()
    
    # removed path function for now
    print("Starting program....")
    
    deriv_dir = "/projects/niblab/bids_projects/Experiments/bro/bids/derivatives"
    main_dict = {}
    run_bash = False
    write_file = False
    sess = "ses-2"
    
    
    
    
    
    if write_file == True:
        ## case: Get all subjects available --add flag for individual subjects or passed list option
        for sub in sub_ids:
            sub_path = os.path.join(deriv_dir,"preprocessed", sub, sess)
            design_file = "pe_design1.fsf"
            # set variables
            task = "pe"
            evs = ['milkshake_cue', 'milkshake_delivery', 'h2O_cue', 'h2O_delivery', 'rinse', 
                  'pe', 'matched_milkshake', 'matched_h2O', 'anticipation_milkshake']
            all_runs = True


            #set_dict(sub)

            if sub not in main_dict:
                main_dict[sub] = {}


            fill_dict(sub,sub_path, main_dict, task, sess, evs, all_runs)
            #print(main_dict['sub-004'])
            make_file(sub, sub_path, sess, main_dict, task, deriv_dir, design_file)
            #make_file(sub, main_dict, task, deriv_dir, fsf_template)
    

    if run_bash == True:
        
        subject_set = sorted([int(x.split("-")[1].lstrip("0")) for x in sub_ids])
        print(subject_set)
        bash_file = os.path.join('/projects/niblab/bids_projects/Experiments/bro/bids/derivatives/code', 'feat1.job')
        start = subject_set[0]
        end = subject_set[-1]
        #print(start, end)
        #for sub_num in subject_set:
        shell_cmd = "sbatch --array={}-{}%{} {}".format(start, end, len(subject_set), bash_file)
        os.system(shell_cmd)
        print(shell_cmd)

            


### Run files

In [33]:
sub_ids = ses2_good_df.index.values
setup_lvl1_design_files(sub_ids)

Starting program....
[4, 22, 25, 26, 27, 28, 29, 30, 32, 33, 36, 38, 39, 40, 41, 44, 45, 46, 47, 52, 53]
sbatch --array=4-53%21 /projects/niblab/bids_projects/Experiments/bro/bids/derivatives/code/feat1.job


In [10]:
!squeue -u nbytes

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)


## Quality Check level 1 models

In [8]:

qc_dict={}
df_dict={}
lvl1_zstat_dict={}
errors=[]

print("[INFO] running quality check...")
for sess in ['ses-1', 'ses-2']:
    for sub_id in ses1_df.index.values:
        sub_path = os.path.join("/projects/niblab/bids_projects/Experiments/bro/bids/derivatives/preprocessed/{}/{}".format(sub_id, sess))
        # set per subject variables

        #sub_id=sub_path.split("/")[-1]

        if sub_id not in qc_dict:
            qc_dict[sub_id] = {}
        if sub_id not in lvl1_zstat_dict:
            lvl1_zstat_dict[sub_id] = {}
        if sess not in lvl1_zstat_dict[sub_id]:
            lvl1_zstat_dict[sub_id][sess] = {}
            
            
        feat1_path = os.path.join(sub_path, "analysis/feat1")
        feat1_folders = sorted(glob.glob(os.path.join(feat1_path, "*.feat")))

        mean_func_ct=0

        for run_folder in feat1_folders:

            # set per run variables

            run_id = run_folder.split("/")[-1].split("_")[1].split(".")[0]
            task_id =  run_folder.split("/")[-1].split("_")[0].split(".")[0]
            
            
            
            zstats=glob.glob(os.path.join(run_folder, 'stats/zstat*nii.gz'))
            filtered_func = os.path.join(run_folder, "filtered_func_data.nii.gz")
            zstat1 = os.path.join(run_folder, "stats/zstat1.nii.gz")
            zstats_ct = len(zstats)
            #print(run_id)


            # CHECK FOR MISSING ZSTATS        
            if not zstats:
                #print("MISSING ZSTATS: {}\t{}".format(sub_path.split("/")[-1], run_id))
                errors.append((sub_id, run_id))
                # ADD REPORT 

            else:
                for zstat_file in zstats:
                    
                    zstat=zstat_file.split("/")[-1].split(".")[0]
                    zstat= "%s_%s_%s"%(task_id, run_id, zstat)
                        
                    cmd='fslstats %s -R'%zstat_file
                    #print(cmd)
                    process = subprocess.Popen(cmd.split(), stdout=subprocess.PIPE)
                    output, error = process.communicate()              
                    output=output.decode("utf-8")
                    output=output.strip("\n")
                    lvl1_zstat_dict[sub_id][sess][zstat] = output
                


            # UPDATE DICTIONARY 
            if run_id == "run-1":
                qc_dict[sub_id]["r1_zstat_ct"] = zstats_ct
            elif run_id == "run-2":
                qc_dict[sub_id]["r2_zstat_ct"] = zstats_ct
      
            #qc_dict[sub_id][run_id]["max_vox"]=max_vox



    print("[INFO] Session: ", sess)
    df = pd.DataFrame(qc_dict).T.fillna(0)
    #display(df)
    print("\n[INFO] Good subjects: \n")
    print(df[(df.T != 0).all()].index.values)
    print("\n[INFO] Subjects missing zstat files: \n")
    print(df[(df.T == 0).any()].index.values)
    df_dict[sess] = df
#for err in errors:
    #check_error_dir(err, deriv_path)


[INFO] running quality check...
[INFO] Session:  ses-1

[INFO] Good subjects: 

['sub-022' 'sub-025' 'sub-026' 'sub-028' 'sub-029' 'sub-030' 'sub-032'
 'sub-033' 'sub-035' 'sub-036' 'sub-038' 'sub-039' 'sub-040' 'sub-041'
 'sub-043' 'sub-044' 'sub-045' 'sub-046' 'sub-047' 'sub-052' 'sub-053']

[INFO] Subjects missing zstat files: 

['sub-001' 'sub-002' 'sub-003' 'sub-004' 'sub-006' 'sub-007' 'sub-008'
 'sub-009' 'sub-010' 'sub-011' 'sub-012' 'sub-013' 'sub-014' 'sub-015'
 'sub-016' 'sub-017' 'sub-018' 'sub-019' 'sub-020' 'sub-021' 'sub-023'
 'sub-027' 'sub-031' 'sub-034' 'sub-037' 'sub-042' 'sub-048' 'sub-049'
 'sub-050' 'sub-051' 'sub-054' 'sub-055' 'sub-056']
[INFO] Session:  ses-2

[INFO] Good subjects: 

['sub-004' 'sub-022' 'sub-025' 'sub-026' 'sub-027' 'sub-028' 'sub-029'
 'sub-030' 'sub-032' 'sub-033' 'sub-035' 'sub-036' 'sub-038' 'sub-039'
 'sub-040' 'sub-041' 'sub-043' 'sub-044' 'sub-045' 'sub-046' 'sub-047'
 'sub-052' 'sub-053']

[INFO] Subjects missing zstat files: 

['sub-0

In [9]:
lvl1_zstat_df =pd.concat({k: pd.DataFrame(v).T for k, v in lvl1_zstat_dict.items()}, axis=0)

/home/nbytes/modules/software/miniconda3/3.0.0/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [23]:
# write dataframe to excel
#lvl1_zstat_df.to_excel('lvl1_zstat_voxel_check.xlsx')


# view level1 zstat voxel check values -- 'fslstats zstat.nii.gz'
tasks=["task-training", "task-pe"]
for task in tasks:
    print("\n[INFO] view zstat voxel check report for feat1 %s models... \n\n"%task)
    display(lvl1_zstat_df.filter(regex=("%s*"%task)))




[INFO] view zstat voxel check report for feat1 task-training models... 




task-training_run-1_zstat1 task-training_run-1_zstat10  \
sub-001 ses-1                        NaN                         NaN   
        ses-2                        NaN                         NaN   
sub-002 ses-1                        NaN                         NaN   
        ses-2                        NaN                         NaN   
sub-003 ses-1                        NaN                         NaN   
        ses-2                        NaN                         NaN   
sub-004 ses-1                        NaN                         NaN   
        ses-2        -3.602966 3.500740          -3.405648 3.178697    
sub-006 ses-1                        NaN                         NaN   
        ses-2                        NaN                         NaN   
sub-007 ses-1                        NaN                         NaN   
        ses-2                        NaN                         NaN   
sub-008 ses-1                        NaN                         NaN   
        ses-2                        NaN                         NaN   
sub-009 ses-1                        NaN                         NaN   
        ses-2                        NaN                         NaN   
sub-010 ses-1                        NaN                         NaN   
        ses-2                        NaN                         NaN   
sub-011 ses-1                        NaN                         NaN   
        ses-2                        NaN                         NaN   
sub-012 ses-1                        NaN                         NaN   
        ses-2                        NaN                         NaN   
sub-013 ses-1                        NaN                         NaN   
        ses-2                        NaN                         NaN   
sub-014 ses-1                        NaN                         NaN   
        ses-2                        NaN                         NaN   
sub-015 ses-1                        NaN                         NaN   
        ses-2                        NaN                         NaN   
sub-016 ses-1                        NaN                         NaN   
        ses-2                        NaN                         NaN   
sub-017 ses-1                        NaN                         NaN   
        ses-2                        NaN                         NaN   
sub-018 ses-1                        NaN                         NaN   
        ses-2                        NaN                         NaN   
sub-019 ses-1                        NaN                         NaN   
        ses-2                        NaN                         NaN   
sub-020 ses-1        -3.557963 3.837220          -4.373131 3.338670    
        ses-2                        NaN                         NaN   
sub-021 ses-1                        NaN                         NaN   
        ses-2                        NaN                         NaN   
sub-022 ses-1        -3.449903 3.479149          -3.800173 4.439355    
        ses-2        -4.908182 5.458394          -4.323241 5.162174    
sub-023 ses-1                        NaN                         NaN   
        ses-2                        NaN                         NaN   
sub-025 ses-1        -3.333854 3.848979          -4.685764 3.897273    
        ses-2        -3.272327 4.215624          -4.634315 3.767376    
sub-026 ses-1        -3.741002 4.385347          -4.198166 5.094502    
        ses-2        -3.569800 4.135463          -3.724078 3.432607    
sub-027 ses-1                        NaN                         NaN   
        ses-2        -5.295704 4.964315          -3.791232 4.805644    
sub-028 ses-1        -3.867210 3.741740          -3.844896 3.766671    
        ses-2        -4.710293 3.957121          -3.677935 3.566500    
sub-029 ses-1        -4.446853 4.504173          -4.163663 6.562389    
        ses-2        -3.679922 4.647163          -3.486717 5.479066    
sub-030 ses-1        -3.730778 5.481168          -4.27


[INFO] view zstat voxel check report for feat1 task-pe models... 




task-pe_run-1_zstat1 task-pe_run-1_zstat10  \
sub-001 ses-1                  NaN                   NaN   
        ses-2                  NaN                   NaN   
sub-002 ses-1                  NaN                   NaN   
        ses-2                  NaN                   NaN   
sub-003 ses-1                  NaN                   NaN   
        ses-2                  NaN                   NaN   
sub-004 ses-1                  NaN                   NaN   
        ses-2  -3.317394 3.548128    -3.544809 3.917557    
sub-006 ses-1                  NaN                   NaN   
        ses-2                  NaN                   NaN   
sub-007 ses-1                  NaN                   NaN   
        ses-2                  NaN                   NaN   
sub-008 ses-1                  NaN                   NaN   
        ses-2                  NaN                   NaN   
sub-009 ses-1                  NaN                   NaN   
        ses-2                  NaN                   NaN   
sub-010 ses-1                  NaN                   NaN   
        ses-2                  NaN                   NaN   
sub-011 ses-1                  NaN                   NaN   
        ses-2                  NaN                   NaN   
sub-012 ses-1                  NaN                   NaN   
        ses-2                  NaN                   NaN   
sub-013 ses-1                  NaN                   NaN   
        ses-2                  NaN                   NaN   
sub-014 ses-1                  NaN                   NaN   
        ses-2                  NaN                   NaN   
sub-015 ses-1                  NaN                   NaN   
        ses-2                  NaN                   NaN   
sub-016 ses-1                  NaN                   NaN   
        ses-2                  NaN                   NaN   
sub-017 ses-1                  NaN                   NaN   
        ses-2                  NaN                   NaN   
sub-018 ses-1                  NaN                   NaN   
        ses-2                  NaN                   NaN   
sub-019 ses-1                  NaN                   NaN   
        ses-2                  NaN                   NaN   
sub-020 ses-1                  NaN                   NaN   
        ses-2                  NaN                   NaN   
sub-021 ses-1                  NaN                   NaN   
        ses-2                  NaN                   NaN   
sub-022 ses-1  -4.098251 4.433403    -3.869772 4.249528    
        ses-2  -4.457385 4.328743    -4.431421 4.502209    
sub-023 ses-1                  NaN                   NaN   
        ses-2                  NaN                   NaN   
sub-025 ses-1  -3.488651 4.770876    -4.458280 4.803546    
        ses-2  -3.696690 3.420614    -3.646660 4.127906    
sub-026 ses-1  -4.561351 3.328633    -3.799060 3.146312    
        ses-2  -4.477532 3.965786    -4.185047 3.849505    
sub-027 ses-1                  NaN                   NaN   
        ses-2  -3.256087 3.540587    -3.695187 3.317549    
sub-028 ses-1  -3.857016 3.860392    -4.010613 3.759566    
        ses-2  -4.218880 4.299455    -4.179132 3.836474    
sub-029 ses-1  -4.001952 3.267989    -3.842397 4.299009    
        ses-2  -3.847484 3.143618    -4.146805 3.987999    
sub-030 ses-1  -3.821234 3.518335    -3.661627 3.522840    
        ses-2  -3.945446 4.332231    -3.113828 3.802166    
sub-031 ses-1                  NaN                   NaN   
        ses-2                  NaN                   NaN   
sub-032 ses-1  -3.618898 4.371154    -3.965327 3.348495    
        ses-2  -3.759617 3.026034    -4.893157 3.842847    
sub-033 ses-1  -4.078511 3.342378    -3.739586 3.679797    
        ses-2  -3.673689 3.863191    -3.917400 3.648266    
sub-034 ses-1                  NaN                   NaN   
        ses-2                  NaN                   NaN   
sub-035 ses-1  -4.680837 3.830943    -4.564849 3.753451    
        ses-2                  NaN                   N

In [33]:
# helper function to color good values green in dataframe
def color_positive_green(val):
    #print(val)
    color = 'green' if val != "0.000000 0.000000 " and pd.isnull(val) != True else 'black'
    #if color == 'red':
     #   print(color)
    #return ['background-color: red' if v else '' for v in is_zero]
    return 'color: %s' % color

In [34]:
# split our dataframes into 2 session dataframes
#lvl1_zstat_df.loc(axis=0)[pd.IndexSlice[:, 'ses-1']]
ses1_zstat = lvl1_zstat_df.xs('ses-1',level=1)
ses2_zstat = lvl1_zstat_df.xs('ses-2', level=1)


In [35]:
## extract good subs here

In [36]:
# separate tasks
train_s2_zstat = ses2_zstat.filter(regex="training")
train_s1_zstat = ses1_zstat.filter(regex="training")
pe_s2_zstat = ses2_zstat.filter(regex="pe")
pe_s1_zstat = ses1_zstat.filter(regex="pe")
# color values
train_s1_zstat=train_s1_zstat.style.applymap(color_positive_green)
train_s2_zstat=train_s2_zstat.style.applymap(color_positive_green)
pe_s1_zstat=pe_s1_zstat.style.applymap(color_positive_green)
pe_s2_zstat=pe_s2_zstat .style.applymap(color_positive_green)

In [66]:
#train_s1_zstat

### Write Files

In [89]:
train_s1_zstat.to_excel("reports/voxel_check.xlsx", sheet_name="ses-1_task-training_lvl1_voxel_check")


In [92]:
#lvl1_zstat_df.to_excel("expanded2.0_lvl1_copes_voxel_check.xlsx")
from openpyxl import load_workbook
    
file_name="/projects/niblab/bids_projects/Experiments/bro/bids/derivatives/code/reports/voxel_check.xlsx"
writer = pd.ExcelWriter(file_name, engine='openpyxl')

if os.path.exists(file_name):
    book = load_workbook(file_name)
    writer.book = book

train_s2_zstat.to_excel(writer, sheet_name="ses-2_task-training_lvl1_voxel_check")
pe_s1_zstat.to_excel(writer, sheet_name="ses-1_task-pe_lvl1_voxel_check")
pe_s2_zstat.to_excel(writer, sheet_name="ses-2_task-pe_lvl1_voxel_check")


writer.save()
writer.close()

## Feat2

### Skipping Registration: 

In [15]:
import glob, os
from shutil import rmtree, copy2
from subprocess import call

In [45]:
base_path='/projects/niblab/bids_projects/Experiments/bro/bids/derivatives/preprocessed' #input("Enter your target study bids path:")
task="task-training"

In [41]:
def skip_registration(task, base_path):
    print('[INFO] skipping registration level1 directory mods for level2...')
    # loop through subjects
    feat_path = 'analysis/feat1/*pe_*.feat'
    for sub_path in glob.glob(os.path.join(base_path,"sub-*" )):
        sub_path = os.path.join(sub_path, "ses-1")
        print("[INFO] modifying subject path %s "%sub_path )
        # get .feat files for subject
        FEATS=glob.glob(os.path.join(sub_path,feat_path))
        # get .mat files for subject
        MAT_DIRS = glob.glob(os.path.join(sub_path, feat_path, "reg/*.mat"))
        # get reg_standard directories for subject
        REGSTD_DIRS = glob.glob(os.path.join(sub_path,feat_path, "reg_standard")) # reg_standard/ directories 

        # remove reg_standard directory
        if not REGSTD_DIRS:
            #print("--------------------------------------->>>> PASSING STEP 1")
            pass
        else:
            for dir_ in REGSTD_DIRS:
                print("[INFO] REMOVING REG_STANDARD DIRECTORY ", dir_)
                rmtree(dir_)

        # remove .mat files
        if not MAT_DIRS:
            #print("--------------------------------------->>>> PASSING STEP 2 PART A")
            pass
        else:
            for mat in MAT_DIRS:
                print("[INFO] REMOVING MAT FILES", mat)
                os.remove(mat)

        # copy identity matrix
        REG_PATHS = glob.glob(os.path.join(sub_path, "analysis/feat1/*.feat/reg"))
        for reg in REG_PATHS:
            path="%s/example_func2standard.mat"%reg
            copy_mat_cmd='cp $FSLDIR/etc/flirtsch/ident.mat %s'%path
            print("[INFO] COPYING IDENTITY MATRIX", copy_mat_cmd)
            os.system(copy_mat_cmd)

        # copy mean files 
        for feat in FEATS:        
            MEAN_PATH = os.path.join(feat, "mean_func.nii.gz")
            REG_DIR = os.path.join(feat, "reg", "standard.nii.gz")
            copy_mean_cmd = 'cp %s %s'%(MEAN_PATH, REG_DIR)
            print("[INFO] COPYING MEAN FILE ", copy_mean_cmd)
            os.system(copy_mean_cmd)






In [ ]:
skip_registration(task, base_path)

### Setup level 2 models

In [87]:
def writeDesignFile(DER_DIR, subID, feat1_folders,tempfsf, task, sess):
    outpath = os.path.join(DER_DIR,"preprocessed", subID,sess, "analysis/feat2/{}_{}_task-{}".format(subID,sess, task) )
    print("[INFO] writing outfiles for ... ",subID )
    tempfsf = tempfsf.replace("OUTPUT", outpath)
    #print(FEATS)
    i = 1
    for run_path in feat1_folders:
        #print(run_path)
        #run = run_path.split("/")[-1].split(".")[0].split("_")[1]
        run = "run%s"%i
        #print("> %s : %s"%(run, run_path))
        tempfsf = tempfsf.replace(run, run_path)
        i += 1
    #print(tempfsf)
    OUTFILE_PATH = os.path.join(DER_DIR, "preprocessed",subID,sess, "analysis/feat2/%s_%s_task-%s_design.fsf"%(subID, sess, task))
    #print("OUTFILE ------------------------>>>> ", OUTFILE_PATH)
    with open(OUTFILE_PATH, "w") as outfile:
        outfile.write(tempfsf)
    outfile.close()


In [88]:

def write_level2_files(task,sess, deriv_dir):
    print("[INFO] writing level2 model design files...")
    for sub_path in glob.glob(os.path.join(base_path, "sub-*" )):

        sub_id = sub_path.split("/")[-1]
        good_runs = 0
        feat1_folders= glob.glob(os.path.join(sub_path, sess,"analysis/feat1/*{}_*.feat".format(task)))

        for folder in feat1_folders:
            zstats=glob.glob(os.path.join(folder, "stats/cope*.nii.gz"))
            #print(zstats)
            if not zstats:
                #print("BAD FOLDER:\n", folder)
                pass
                #rmtree(folder)
            else:
                good_runs += 1

        # start analysis

        ### check for existence of feat2 directory
        FEAT2_DIR = os.path.join(sub_path,sess, "analysis/feat2")
        #print(FEAT2_DIR)
        if os.path.exists(FEAT2_DIR):
            pass
        else:
            os.makedirs(FEAT2_DIR)

        if good_runs == 2:
            with open(os.path.join(deriv_dir, "code/design_file_templates/design2_pe.fsf"), 'r') as infile:
                tempfsf=infile.read()
                writeDesignFile(deriv_dir, sub_id, feat1_folders, tempfsf, task, sess)
                infile.close()
        else:
            pass



In [90]:
deriv_dir='/projects/niblab/bids_projects/Experiments/bro/bids/derivatives'
sess='ses-1'
task = 'pe'

write_level2_files(task,sess,deriv_dir)

[INFO] writing level2 model design files...
[INFO] writing outfiles for ...  sub-025
[INFO] writing outfiles for ...  sub-028
[INFO] writing outfiles for ...  sub-029
[INFO] writing outfiles for ...  sub-030
[INFO] writing outfiles for ...  sub-032
[INFO] writing outfiles for ...  sub-033
[INFO] writing outfiles for ...  sub-036
[INFO] writing outfiles for ...  sub-038
[INFO] writing outfiles for ...  sub-039
[INFO] writing outfiles for ...  sub-040
[INFO] writing outfiles for ...  sub-041
[INFO] writing outfiles for ...  sub-045
[INFO] writing outfiles for ...  sub-046
[INFO] writing outfiles for ...  sub-047
[INFO] writing outfiles for ...  sub-053
[INFO] writing outfiles for ...  sub-035
[INFO] writing outfiles for ...  sub-022
[INFO] writing outfiles for ...  sub-026
[INFO] writing outfiles for ...  sub-043


### Submit batch jobs

In [91]:
def submit_slurm(sess, task, bash_file, run_cmd=False):
    print("[INFO] submitting slurm job...")
    
    df = df_dict[sess]
    good_subs = df[(df.T!=0).all()].index.values
    subject_set = sorted([int(x.split("-")[1].lstrip("0")) for x in good_subs])
    #print(subject_set)
    
    start = subject_set[0]
    end = subject_set[-1]
    #print(start, end)
    #for sub_num in subject_set:
    shell_cmd = "sbatch --array={}-{}%{} {} {} {}".format(start, end, len(subject_set), bash_file, sess, task)
    if run_cmd == True:
        print("[INFO] submitted job")
        os.system(shell_cmd)
    print("[INFO] BATCH COMMAND: ", shell_cmd)

In [94]:
sess='ses-2'
task = 'pe'

bash_file =os.path.join('/projects/niblab/bids_projects/Experiments/bro/bids/derivatives/code', 'feat2.job')
submit_slurm(sess,task, bash_file, run_cmd=True)

[INFO] submitting slurm job...
[INFO] submitted job
[INFO] BATCH COMMAND:  sbatch --array=4-53%23 /projects/niblab/bids_projects/Experiments/bro/bids/derivatives/code/feat2.job ses-2 pe


In [95]:
!squeue -u nbytes

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
2950356_[42-53%23]     batch    f2bro   nbytes PD       0:00      1 (JobArrayTaskLimit)
        2950356_29     batch    f2bro   nbytes  R       0:01      1 compute-7-9
        2950356_30     batch    f2bro   nbytes  R       0:01      1 compute-0-12
        2950356_32     batch    f2bro   nbytes  R       0:01      1 compute-0-15
        2950356_33     batch    f2bro   nbytes  R       0:01      1 compute-0-23
        2950356_36     batch    f2bro   nbytes  R       0:01      1 compute-0-24
        2950356_38     batch    f2bro   nbytes  R       0:01      1 compute-4-26
        2950356_39     batch    f2bro   nbytes  R       0:01      1 compute-0-19
        2950356_40     batch    f2bro   nbytes  R       0:01      1 compute-0-28
        2950356_41     batch    f2bro   nbytes  R       0:01      1 compute-4-31
         2950356_4     batch    f2bro   nbytes  R       0:02      1 largemem-0-0
        2950356_25

### Quality Check 

In [96]:
# helper function to generate qc level2 report
def qc_level2(base_path, task):
    for sess in ['ses-1', 'ses-2']:
        for sub_id in ses1_df.index.values:

            sub_dir = os.path.join(base_path, sub_id, sess)

            if sub_id not in qc_dict:
                qc_dict[sub_id] = {}
            if sub_id not in lvl2_zstat_dict:
                lvl2_zstat_dict[sub_id] = {}
            if sess not in lvl2_zstat_dict[sub_id]:
                lvl2_zstat_dict[sub_id][sess] = {}



            feat2_folder=os.path.join(sub_dir, 'analysis/feat2/{}_{}_{}.gfeat'.format(sub_id,sess, task))
            mean_func = os.path.join(feat2_folder,'mean_func.nii.gz')
            cope_folders= glob.glob(os.path.join(feat2_folder, "*.feat"))
            cope_folder_ct = len(cope_folders)
            zstat_files = glob.glob(os.path.join(feat2_folder, "*.feat/stats/zstat1.nii.gz"))
            zstat_file_ct = len(zstat_files)

            qc_dict[sub_id]["zstat_file_ct"] = zstat_file_ct
            qc_dict[sub_id]["cope_folder_ct"] = cope_folder_ct



            for zstat in sorted(zstat_files):
                cope=zstat.split("/")[-3].split(".")[0]
                #print(cope)
                cmd='fslstats %s -R'%zstat

                process = subprocess.Popen(cmd.split(), stdout=subprocess.PIPE)
                output, error = process.communicate()              

                output=output.decode("utf-8")
                output=output.strip("\n")
                lvl2_zstat_dict[sub_id][sess][cope] = output





        df=pd.DataFrame(qc_dict).T
        print("[INFO] session: %s \t%s "%(sess,task))
        print("[INFO] good subject count: %s \n"%len(df[(df.T !=0).all()].index.values))
        #display(df)
        lvl2_df_dict[sess] = df 


In [97]:
qc_dict={}
lvl2_df_dict ={}
mean_func_ct=0
lvl2_zstat_dict={}
subjects_with_0s = []
task="task-pe"
qc_level2(base_path,task="task-pe")

[INFO] session: ses-1 	task-pe 
[INFO] good subject count: 19 

[INFO] session: ses-2 	task-pe 
[INFO] good subject count: 19 



In [98]:
lvl2_zstat_df =pd.concat({k: pd.DataFrame(v).T for k, v in lvl2_zstat_dict.items()}, axis=0, sort=True)
#lvl2_zstat_df = lvl2_zstat_df.fillna(0)


In [100]:
print("[INFO] %s : zstat voxel check report..."%task)
lvl2_zstat_df.style.applymap(color_positive_green)

[INFO] task-pe : zstat voxel check report...


### Setup level 3 models

In [156]:
# helper function to write model3 design files
def write_lvl3_designs(folder, template, sub_ct, cope_ct, cope_dict, sess, task):
    deriv_dir = folder
    template_file = template
    # get the number of cope files to make (# of copes)
    num_of_copes = cope_ct
    num_of_input = sub_ct
    #print(num_of_copes)
    
    # loop through copes and make design file for each
    for cope_num in range(1, num_of_copes+1):
        
        if cope_num not in cope_dict:
            cope_dict[cope_num] = {}
            
        #OUTPUTDIR = os.path.join(deriv_dir, 'group_ana/cope%s_ses-1'%cope_num)
        OUTPUTDIR = os.path.join(deriv_dir, 'preprocessed/group_analysis/%s/%s_task-%s_cope%s'%(task.split('-')[1],sess,task, cope_num))

        COPES = glob.glob(os.path.join(deriv_dir, "preprocessed", "sub-*",sess,'analysis/feat2/sub-*%s*gfeat/cope%s.feat'%(task,cope_num)))
        COPES = sorted(COPES)
        
        
        for x,cope in enumerate(COPES):
            
            count=int(x)+1
            if count > 9:
                INPUTX = "INPUT_%i"%(count)
            else:
                INPUTX = "INPUT%i"%(count)
            cope_dict[cope_num][INPUTX] = cope
            #print("%s >>>>-----> %s"%(INPUTX,cope))
        with open(template_file, 'r') as infile:
            tempfsf=infile.read()
            tempfsf = tempfsf.replace("OUTPUT", OUTPUTDIR)
            for input_title in sorted(cope_dict[cope_num]):
                input_ = cope_dict[cope_num][input_title]
                tempfsf = tempfsf.replace("%s"%input_title, input_)
            OUTFILE_PATH = os.path.join(deriv_dir, 'preprocessed/group_analysis/%s/%s_%s_cope%s.fsf'%(task.split('-')[1],sess,task,cope_num))
            #print(tempfsf)
            print("[INFO] writing output file...", OUTFILE_PATH)
            if not os.path.exists(os.path.join(deriv_dir, 'preprocessed/group_analysis/%s'%task.split('-')[1])):
                os.makedirs(os.path.join(deriv_dir, 'preprocessed/group_analysis/%s'%task.split('-')[1]))
                
            with open(OUTFILE_PATH, 'w') as outfile:
                outfile.write(tempfsf)
            outfile.close()
        infile.close()




In [151]:

def write_design_files():
    
    sess = 'ses-1'
    task='task-pe'
    cope_dict = {}
    base_folder = '/projects/niblab/bids_projects/Experiments/bro/bids/derivatives'
    template_fsf = '/projects/niblab/bids_projects/Experiments/bro/bids/derivatives/code/model_designs/design3.19.fsf'
    sub_ct = 19
    cope_ct = 17
    
    
    print("[INFO] ses: %s subject count: %s task: %s \tcope count: %s"%(sess,sub_ct, task,cope_ct))
    write_lvl3_designs(base_folder, template_fsf, sub_ct, cope_ct, cope_dict,sess, task)
    
    
cope_ct=17

write_design_files()



[INFO] ses: ses-1 subject count: 19 task: task-pe 	cope count: 17
[INFO] writing output file... /projects/niblab/bids_projects/Experiments/bro/bids/derivatives/preprocessed/group_analysis/pe/ses-1_task-pe_cope1.fsf
[INFO] writing output file... /projects/niblab/bids_projects/Experiments/bro/bids/derivatives/preprocessed/group_analysis/pe/ses-1_task-pe_cope2.fsf
[INFO] writing output file... /projects/niblab/bids_projects/Experiments/bro/bids/derivatives/preprocessed/group_analysis/pe/ses-1_task-pe_cope3.fsf
[INFO] writing output file... /projects/niblab/bids_projects/Experiments/bro/bids/derivatives/preprocessed/group_analysis/pe/ses-1_task-pe_cope4.fsf
[INFO] writing output file... /projects/niblab/bids_projects/Experiments/bro/bids/derivatives/preprocessed/group_analysis/pe/ses-1_task-pe_cope5.fsf
[INFO] writing output file... /projects/niblab/bids_projects/Experiments/bro/bids/derivatives/preprocessed/group_analysis/pe/ses-1_task-pe_cope6.fsf
[INFO] writing output file... /projects/

In [152]:
def run_lvl3_models(task,sess, cope_ct, run_cmd=False):
    task=task.split('-')[1]
    sess=sess.split('-')[1]
    
    # for ses-1
    bash_file = os.path.join('/projects/niblab/bids_projects/Experiments/bro/bids/derivatives/code', 'feat3.job') 
    start = 1
    end = cope_ct

    shell_cmd = "sbatch --array={}-{}%{} {} {} {}".format(start, end, end, bash_file, sess, task)
    if run_cmd == True:
        os.system(shell_cmd)
    print("[INFO] running command ... %s"%shell_cmd)
    



In [154]:
cope_ct=17
sess="ses-2"
task="task-pe"
run_lvl3_slurm(task,sess, cope_ct, True)

[INFO] running command ... sbatch --array=1-17%17 /projects/niblab/bids_projects/Experiments/bro/bids/derivatives/code/feat3.job 2 pe


In [155]:
!squeue -u nbytes

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
         2950475_1     batch    F3bro   nbytes  R       0:10      1 compute-0-0
         2950475_2     batch    F3bro   nbytes  R       0:10      1 compute-0-1
         2950475_3     batch    F3bro   nbytes  R       0:10      1 compute-0-3
         2950475_4     batch    F3bro   nbytes  R       0:10      1 compute-0-4
         2950475_5     batch    F3bro   nbytes  R       0:10      1 compute-0-6
         2950475_6     batch    F3bro   nbytes  R       0:10      1 compute-0-7
         2950475_7     batch    F3bro   nbytes  R       0:10      1 compute-0-11
         2950475_8     batch    F3bro   nbytes  R       0:10      1 compute-0-12
         2950475_9     batch    F3bro   nbytes  R       0:10      1 compute-0-15
        2950475_10     batch    F3bro   nbytes  R       0:10      1 compute-0-19
        2950475_11     batch    F3bro   nbytes  R       0:10      1 compute-0-23
        2950475_12     batch  

### Quality Check level 3

In [157]:

def qc_lvl3_models(task):
    task=task.split("-")[1]
    for cope_dir in glob.glob('/projects/niblab/bids_projects/Experiments/bro/bids/derivatives/preprocessed/group_analysis/%s/*.gfeat'%task):
        #es-2_task-training_cope10.gfeat
        sess_id=cope_dir.split("/")[-1].split(".")[0].split("_")[0]
        cope_id=cope_dir.split("/")[-1].split(".")[0].split("_")[2]
        if cope_id not in qc_dict:
            qc_dict[cope_id] = {}
        #print(cope_id)
        if sess_id not in lvl3_dict:
            lvl3_dict[sess_id] = {}
        if cope_id not in lvl3_dict[sess_id]:
            lvl3_dict[sess_id][cope_id] = {}


        stats_folder=glob.glob(os.path.join(cope_dir, 'cope1.feat/stats/*'))
        if not stats_folder:
            print("MISSING")

        zstat_file_ct = len(stats_folder)

        qc_dict[cope_id]["zstat_file_ct"] = zstat_file_ct

        zstat_files = glob.glob(os.path.join(cope_dir, "cope1.feat/stats/zstat1.nii.gz"))
        zstat_file_ct = len(zstat_files)

        for zstat in sorted(zstat_files):
            cope=zstat.split("/")[-4].split(".")[0]
            #print(cope)

            cmd='fslstats %s -R'%zstat
            #voxel_intensity = stats.run()
            #output = list(stats.aggregate_outputs()
            process = subprocess.Popen(cmd.split(), stdout=subprocess.PIPE)
            output, error = process.communicate()              

            output=output.decode("utf-8")
            output=output.strip("\n")

            lvl3_dict[sess_id][cope_id]['zstat'] = output


In [162]:

for task in ['task-pe', 'task-training']:
    mean_func_ct=0
    qc_dict={}
    lvl3_dict={}
    qc_lvl3_models(task)
    print("[INFO] %s \n[INFO] Voxel check from zstat files in the level 3 models"%(task))
    lvl3_zstat_df =pd.concat({k: pd.DataFrame(v).T for k, v in lvl3_dict.items()}, axis=0, sort=True)
    display(lvl3_zstat_df.T)

[INFO] task-pe 
[INFO] Voxel check from zstat files in the level 3 models


ses-1                                            \
                    cope15               cope16               cope17   
zstat  -6.852843 5.401256   -3.081025 3.459982   -2.939416 3.236789    

                                                                      \
                    cope13               cope14                cope1   
zstat  -2.829524 2.652389   -3.321151 2.687091   -3.279552 3.109510    

                                                                      \
                     cope2                cope3                cope4   
zstat  -3.213049 3.069424   -3.794251 3.813819   -2.653368 2.829780    

                                                                      \
                     cope6                cope9                cope8   
zstat  -3.106179 3.186391   -2.653368 3.313399   -3.186391 3.106179    

                                                                      \
                     cope5               cope10                cope7   
zstat  -5.643332 5.208662   -3.774555 3.777668   -3.313399 2.653368    

                                                               ses-2  \
                    cope11               cope12                cope1   
zstat  -2.830132 2.654551   -2.889757 3.431960   -2.822291 3.465023    

                                                                      \
                     cope2                cope3                cope4   
zstat  -2.919338 3.543251   -3.056975 3.786303   -3.070341 3.166232    

                                                                      \
                     cope5                cope6                cope7   
zstat  -5.491108 3.898543   -2.811543 3.589213   -3.166231 3.070340    

                                                                      \
                     cope8                cope9               cope10   
zstat  -3.589213 2.811543   -3.070340 3.166231   -2.975651 3.945512    

                                                                      \
                    cope11               cope12               cope13   
zstat  -3.166168 3.070173   -3.070332 3.166228   -3.166547 3.070489    

                                                                      \
                    cope14               cope15               cope16   
zstat  -3.166198 3.070252   -8.283455 4.488976   -3.070317 3.166216    

                            
                    cope17  
zstat  -3.234330 5.718135

[INFO] task-training 
[INFO] Voxel check from zstat files in the level 3 models


ses-1                                            \
                    cope10                cope1                cope2   
zstat  -3.539954 5.231525   -4.002584 5.274480   -4.290170 3.583143    

                                                                      \
                     cope3                cope4                cope5   
zstat  -3.799652 5.483512   -4.685711 2.883127   -4.988431 5.323785    

                                                                      \
                     cope6                cope7                cope8   
zstat  -4.024570 3.257300   -3.407811 4.069576   -3.257300 4.024570    

                                          ses-2                       \
                     cope9               cope10                cope1   
zstat  -4.069576 3.407811   -3.680752 4.597322   -3.729415 4.385585    

                                                                      \
                     cope2                cope3                cope4   
zstat  -3.914823 3.175563   -3.938979 4.411117   -3.435974 3.141903    

                                                                      \
                     cope5                cope6                cope7   
zstat  -4.380487 4.930459   -3.215672 3.609460   -3.337725 3.250712    

                                                 
                     cope8                cope9  
zstat  -3.609460 3.215672   -3.250712 3.337725